Testes de hipóteses

In [2]:
# instalar versão atualizada
! sudo pip install scipy==1.7.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 61.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
arviz 0.15.1 requires scipy>=1.8.0, but you have scipy 1.7.3 which is incompatible.
chex 0.1.85 requires numpy>=1.24.1, but you have numpy 1.22.4 which is incompatible.
jax 0.4.23 requires scipy>=1.9, but you have scipy 1.7.3 which is incompatible.
jaxlib 0.4.23+cuda12.cudnn89 requires scipy>=1.9, but you have scipy 1.7.3 which is incompatible.
plotnine 0.12.

In [3]:
# instalar nova ferramenta para testes de hipoteses

! sudo pip install pingouin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 5.2 MB/s eta 0:00:00


In [5]:
# modulos
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
# analises
from scipy import stats as st
import pingouin as pg

In [13]:
dados_ab = pd.read_csv('exampleDataABtest.csv')
dados_ab.head()

,group,time,clickedTrue
0,A,2016-06-02 02:17:53,0
1,A,2016-06-02 03:03:54,0
2,A,2016-06-02 03:18:56,1
3,B,2016-06-02 03:23:43,0
4,A,2016-06-02 04:04:00,0


In [20]:
dados_nps = pd.read_csv('nps_example.csv', sep = ';')
dados_nps.head()

,id,response_status,how_long_listening,age,nps_score,gender
0,11706300,Complete,Less than 6 months,25-34,10.0,Female
1,11706302,Complete,1 year to less than 3 years,25-34,10.0,Female
2,11706307,Complete,6 months to less than a year,35-44,10.0,Female
3,11706312,Complete,Less than 6 months,35-44,10.0,Female
4,11706316,Complete,6 months to less than a year,25-34,10.0,Male


###Teste T

- comparar os grupos

In [14]:
dados_ab.groupby('group').agg(media_cliques = pd.NamedAgg('clickedTrue', 'mean'),
                              dp_cliques = pd.NamedAgg('clickedTrue', 'std'),
                              n = pd.NamedAgg('clickedTrue', 'count'))

,media_cliques,dp_cliques,n
group,,,
A,0.04,0.196155,500
B,0.08,0.271565,500


In [16]:
grA = dados_ab[dados_ab['group'] == 'A']['clickedTrue']
grB = dados_ab[dados_ab['group'] == 'B']['clickedTrue']

In [18]:
# comparar que grupo A é diferente do grupo B
st.ttest_ind(a=grA, b=grB, alternative='two-sided')

Ttest_indResult(statistic=-2.669938469060931, pvalue=0.007709783987515948)

In [ ]:
# se o p-value < 0,05 - entao rejeito a hipotese nula

In [19]:
# teste t
pg.ttest(x=grA,
         y=grB,
         alternative='two-sided',
         confidence = 0.95)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-2.669938,998,two-sided,0.00771,"[-0.07, -0.01]",0.168862,2.349,0.760344


In [ ]:
# se o p-value < 0,05 - entao rejeito a hipotese nula

##Teste F##

In [23]:
#perfil das respostas
dados_nps.groupby('response_status') \
  .size()\
  .to_frame('n')\
  .reset_index()

,response_status,n
0,Complete,2281
1,Incomplete,265
2,Terminated,33


In [24]:
# verificar nulos
dados_nps[dados_nps['nps_score'].isnull()]

,id,response_status,how_long_listening,age,nps_score,gender
17,11706467,Incomplete,Less than 6 months,18-24,NaN,NaN
31,11706938,Incomplete,1 year to less than 3 years,25-34,NaN,NaN
32,11706979,Incomplete,6 months to less than a year,25-34,NaN,NaN
43,11707426,Incomplete,6 months to less than a year,25-34,NaN,NaN
48,11707719,Incomplete,3 years to less than 5 years,35-44,NaN,NaN
...,...,...,...,...,...,...
2546,13093216,Incomplete,6 months to less than a year,35-44,NaN,NaN
2556,13278063,Incomplete,3 years to less than 5 years,18-24,NaN,NaN
2570,13565327,Complete,1 year to less than 3 years,45-54,NaN,Female
2572,13601847,Incomplete,3 years to less than 5 years,25-34,NaN,NaN


In [25]:
# filtrar dados
dados_nps_filtrados = dados_nps[(dados_nps['response_status'] == 'Complete') & \
                                (dados_nps['nps_score'].notna())]

In [26]:
dados_nps_filtrados.head()

,id,response_status,how_long_listening,age,nps_score,gender
0,11706300,Complete,Less than 6 months,25-34,10.0,Female
1,11706302,Complete,1 year to less than 3 years,25-34,10.0,Female
2,11706307,Complete,6 months to less than a year,35-44,10.0,Female
3,11706312,Complete,Less than 6 months,35-44,10.0,Female
4,11706316,Complete,6 months to less than a year,25-34,10.0,Male


In [27]:
# resumo dados
dados_nps_filtrados.groupby('age') \
                   .agg(media_nps = pd.NamedAgg('nps_score', 'mean'),
                        dp_nps = pd.NamedAgg('nps_score', 'std'),
                        n = pd.NamedAgg('nps_score', 'size')) \
                   .reset_index()

,age,media_nps,dp_nps,n
0,18-24,9.464539,1.116275,282
1,25-34,9.694828,0.957639,580
2,35-44,9.707612,0.979501,578
3,45-54,9.719039,0.928254,541
4,55-64,9.733871,0.923020,248
5,65-74,9.423077,1.361560,26
6,75+,8.000000,0.000000,2


In [28]:
# filtrar 75+
dados_nps_filtrados_aj = dados_nps_filtrados[dados_nps_filtrados['age'] != '75+']

In [30]:
dados_nps_filtrados_aj.groupby('age') \
                   .agg(media_nps = pd.NamedAgg('nps_score', 'mean'),
                        dp_nps = pd.NamedAgg('nps_score', 'std'),
                        n = pd.NamedAgg('nps_score', 'size')) \
                   .reset_index()

,age,media_nps,dp_nps,n
0,18-24,9.464539,1.116275,282
1,25-34,9.694828,0.957639,580
2,35-44,9.707612,0.979501,578
3,45-54,9.719039,0.928254,541
4,55-64,9.733871,0.923020,248
5,65-74,9.423077,1.361560,26


Teste scipy

In [31]:
# separar os grupos
dados_18_24 = dados_nps_filtrados_aj[dados_nps_filtrados_aj['age'] == '18-24']['nps_score']
dados_25_34 = dados_nps_filtrados_aj[dados_nps_filtrados_aj['age'] == '25-34']['nps_score']
dados_35_44 = dados_nps_filtrados_aj[dados_nps_filtrados_aj['age'] == '35-44']['nps_score']
dados_45_54 = dados_nps_filtrados_aj[dados_nps_filtrados_aj['age'] == '45-54']['nps_score']
dados_55_64 = dados_nps_filtrados_aj[dados_nps_filtrados_aj['age'] == '55-64']['nps_score']
dados_65_74 = dados_nps_filtrados_aj[dados_nps_filtrados_aj['age'] == '65-74']['nps_score']
# realizar teste
st.f_oneway(dados_18_24,
            dados_25_34,
            dados_35_44,
            dados_45_54,
            dados_55_64,
            dados_65_74)

F_onewayResult(statistic=3.5221660981040768, pvalue=0.0035606861304276695)

In [ ]:
## se o valor p < 0.05, rejeitamos a hiposete nula

In [32]:
# teste f
pg.anova(dv = 'nps_score',
         between = 'age',
         data = dados_nps_filtrados_aj,
         detailed = True)

,Source,SS,DF,MS,F,p-unc,np2
0,age,16.888794,5,3.377759,3.522166,0.003561,0.00777
1,Within,2156.791916,2249,0.959000,NaN,NaN,NaN


In [ ]:
## se o valor p < 0.05, rejeitamos a hiposete nula